<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/opmnt_library_transformer_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初期化

## インストール

In [3]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/.shortcut-targets-by-id/1liJWbv2EhngAj7pixTjzRpv9B-Bz9zCt/Colab Notebooks


In [4]:
ls

data@                                      pred.txt
new_colab2.ipynb                           toy-ende@
opmnt_library_test.ipynb                   Transformer@
opmnt_library_transformer_translate.ipynb  transformer_kyoto_ja_en.ipynb
packages@                                  transformer_test.ipynb


In [5]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール

In [6]:
pip install OpenNMT-py

     |████████████████████████████████| 216 kB 6.6 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 16.0 MB 117 kB/s 
     |████████████████████████████████| 1.2 MB 37.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


## インポート

In [7]:
import yaml
import torch
import torch.nn as nn
from argparse import Namespace
from collections import defaultdict, Counter

In [8]:
import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields, IterOnDevice
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.utils.misc import set_random_seed

In [9]:
# enable logging
from onmt.utils.logging import init_logger, logger
init_logger()

<RootLogger root (INFO)>

In [10]:
src_vocab_path = "data/kftt-data-1.0/run/example.vocab.src"
tgt_vocab_path = "data/kftt-data-1.0/run/example.vocab.tgt"

In [11]:
# initialize the frequency counter
counters = defaultdict(Counter)
# load source vocab
_src_vocab, _src_vocab_size = _load_vocab(
    src_vocab_path,
    'src',
    counters)
# load target vocab
_tgt_vocab, _tgt_vocab_size = _load_vocab(
    tgt_vocab_path,
    'tgt',
    counters)

[2021-11-06 13:51:31,696 INFO] Loading src vocabulary from data/kftt-data-1.0/run/example.vocab.src
[2021-11-06 13:51:32,857 INFO] Loaded src vocab has 13637 tokens.
[2021-11-06 13:51:32,867 INFO] Loading tgt vocabulary from data/kftt-data-1.0/run/example.vocab.tgt
[2021-11-06 13:51:33,071 INFO] Loaded tgt vocab has 12199 tokens.


In [12]:
# initialize fields
src_nfeats, tgt_nfeats = 0, 0 # do not support word features for now
fields = get_fields(
    'text', src_nfeats, tgt_nfeats)

In [13]:
# build fields vocab
share_vocab = False
vocab_size_multiple = 1
src_vocab_size = 30000
tgt_vocab_size = 30000
src_words_min_frequency = 1
tgt_words_min_frequency = 1
vocab_fields = _build_fields_vocab(
    fields, counters, 'text', share_vocab,
    vocab_size_multiple,
    src_vocab_size, src_words_min_frequency,
    tgt_vocab_size, tgt_words_min_frequency)

[2021-11-06 13:51:33,145 INFO]  * tgt vocab size: 12203.
[2021-11-06 13:51:33,171 INFO]  * src vocab size: 13639.


In [14]:
src_val = "data/kftt-data-1.0/data/tok/kyoto-test.en"
tgt_val = "data/kftt-data-1.0/data/tok/kyoto-test.ja"

In [15]:
"""
src_data = {"reader": onmt.inputters.str2reader["text"](), "data": src_val}
tgt_data = {"reader": onmt.inputters.str2reader["text"](), "data": tgt_val}
_readers, _data = onmt.inputters.Dataset.config(
    [('src', src_data), ('tgt', tgt_data)])
"""
src_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": src_val,
    "features": {}
}
tgt_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": tgt_val,
    "features": {}
}
_readers, _data = onmt.inputters.Dataset.config(
    [("src", src_data), ("tgt", tgt_data)]
)

In [16]:
dataset = onmt.inputters.Dataset(
    vocab_fields, readers=_readers, data=_data,
    sort_key=onmt.inputters.str2sortkey["text"])

In [17]:
data_iter = onmt.inputters.OrderedIterator(
            dataset=dataset,
            device="cuda",
            batch_size=10,
            train=False,
            sort=False,
            sort_within_batch=True,
            shuffle=False
        )

In [18]:
src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields['tgt'].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [19]:
emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding)

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings)

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding)
decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1)).to(device)

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator)

In [20]:
src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]
scorer = GNMTGlobalScorer(alpha=0.7, 
                          beta=0., 
                          length_penalty="avg", 
                          coverage_penalty="none")
gpu = 0 if torch.cuda.is_available() else -1
translator = Translator(model=model, 
                        fields=vocab_fields, 
                        src_reader=src_reader, 
                        tgt_reader=tgt_reader, 
                        global_scorer=scorer,
                        gpu=gpu)
builder = onmt.translate.TranslationBuilder(data=dataset, 
                                            fields=vocab_fields)

In [71]:
import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
#from onmt.opts import dynamic_prepare_opts

parser = ArgumentParser(description='translate.py')
#dynamic_prepare_opts(parser, build_vocab_only=True)
opts.translate_opts(parser)

base_args = (["-model", "data/kftt-data-1.0/run/model_step_15000.pt", "-src", "data/kftt-data-1.0/data/tok/kyoto-test.en", "-output", "data/kftt-data-1.0/out.txt", "-gpu", "0", "-verbose"])
opt, unknown = parser.parse_known_args(base_args)
opt

Namespace(align_debug=False, alpha=0.0, attn_debug=False, avg_raw_probs=False, ban_unk_token=False, batch_size=30, batch_type='sents', beam_size=5, beta=-0.0, block_ngram_repeat=0, coverage_penalty='none', data_type='text', dump_beam='', fp32=False, gpu=0, ignore_when_blocking=[], int8=False, length_penalty='none', log_file='', log_file_level='0', max_length=100, max_sent_length=None, min_length=0, models=['data/kftt-data-1.0/run/model_step_15000.pt'], n_best=1, output='data/kftt-data-1.0/out.txt', phrase_table='', random_sampling_temp=1.0, random_sampling_topk=0, random_sampling_topp=0.0, ratio=-0.0, replace_unk=False, report_align=False, report_time=False, seed=-1, shard_size=10000, src='data/kftt-data-1.0/data/tok/kyoto-test.en', src_feats=None, stepwise_penalty=False, tgt=None, tgt_prefix=False, verbose=True)

In [74]:
from onmt.utils.parse import ArgumentParser
import onmt.inputters as inputters
from onmt.utils.misc import use_gpu

model_path="data/kftt-data-1.0/run/model_step_15000.pt"

checkpoint = torch.load(model_path,map_location=lambda storage, loc: storage)

model_opt = ArgumentParser.ckpt_model_opts(checkpoint['opt'])
ArgumentParser.update_model_opts(model_opt)
ArgumentParser.validate_model_opts(model_opt)
fields = checkpoint['vocab']

# Avoid functionality on inference
model_opt.update_vocab = False

model = onmt.model_builder.build_base_model(model_opt, fields, use_gpu(opt), checkpoint,
                            opt.gpu)
if opt.fp32:
    model.float()
elif opt.int8:
    if opt.gpu >= 0:
        raise ValueError(
            "Dynamic 8-bit quantization is not supported on GPU")
    torch.quantization.quantize_dynamic(model, inplace=True)
model.eval()
model.generator.eval()

Sequential(
  (0): Linear(in_features=512, out_features=12203, bias=True)
  (1): Cast()
  (2): LogSoftmax(dim=-1)
)

In [75]:
model

NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(13639, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, o

In [76]:
from onmt.constants import DefaultTokens
from onmt.utils.misc import tile, set_random_seed, report_matrix

# アテンションデバッグをON
attn_debug=True
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=[src_vocab],
        attn_debug=attn_debug)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
        
        if attn_debug:
            preds = trans.pred_sents[0]
            preds.append(DefaultTokens.EOS)
            attns = trans.attns[0].tolist()
            if translator.data_type == "text":
                srcs = trans.src_raw
            else:
                srcs = [str(item) for item in range(len(attns[0]))]
            output = report_matrix(srcs, preds, attns)
            if translator.logger:
                translator.logger.info(output)
            else:
                #os.write(1, output.encode("utf-8"))
                print(output)
    break


SENT 0: ['Infobox', 'Buddhist']
PRED 0: 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同 合同
PRED SCORE: -9.1529

              Infobox    Buddhis 
        合同  0.4886951 *0.5113049 
        合同  0.4928581 *0.5071419 
        合同  0.4972830 *0.5027170 
        合同 *0.5024315  0.4975685 
        合同 *0.5067246  0.4932754 
        合同 *0.5098452  0.4901548 
        合同 *0.5119579  0.4880421 
        合同 *0.5133317  0.4866683 
        合同 *0.5142041  0.4857959 
        合同 *0.5147507  0.4852493 
        合同 *0.5150910  0.4849090 
        合同 *0.5153027  0.4846974 
        合同 *0.5154346  0.4845654 
        合同 *0.5155176  0.4844825 
        合同 *0.5155701  0.4844299 
        合同 *0.5156039  0.4843961 
        合同 *0.5156258  0.4843742 
        合同 *0.5156403  0.4

# アテンションの利用

In [23]:
import onmt
import onmt.inputters
import onmt.translate
import onmt.model_builder
from collections import namedtuple

# Load the model.
Opt = namedtuple('Opt', ['model', 'data_type', 'reuse_copy_attn', "gpu"])

opt = Opt("/Users/esalesky/projects/models/lstm_clean_acc_51.81_ppl_15.58_e13.pt", "text",False, 0)
